# Backpropagation com Gradiente descendente modular

# Introdução

Neste tutorial vamos derivar um algoritmo de backpropagation que será utilizado para construir e treinar redes neurais. Para manter a simplicidade, vou explicar esse algoritmo num contexto de regressão linear, mas será fácil ver como ele generaliza para as redes neurais. 

Em primeiro lugar, nós vamos representar o modelo de regressão linear com uma arquitetura de camadas. Então, nós veremos o que cada camada precisa fazer para que seja possível minimizar o custo do nosso modelo. Já adiantando, nós vamos ver que cada camada precisará (1) computar uma função que levará de um input até um output, (2) ter um mecanismo para propagar uma derivada para trás e (3) precisará computar a derivada do output da camada com respeito aos parâmetros da camada. Com essas três operações, nós conseguiremos saber as derivadas da função custo com respeito a qualquer parâmetro do modelo. Então podemos aplicar Gradiente Descendente Estocástico e otimizar a função custo iterativamente.

Esse tutorial é bastante intensivo em matemática, mas você não precisa entender todos os mínimos detalhes. Eu chamarei atenção quando for falar de algo importante, que necessita mais da sua atenção. Por exemplo, é **MUITO IMPORTANTE** que, ao final desse tutorial, você entenda que **nós não implementamos redes neurais inteiras; nós implementamos as suas camadas!** Guarde isso desde já e mantenha isso em menta até o final dessa leitura.


# Arquitetura por camadas de regressão linear

Para entender o algoritmo de backpropagation, é útil especificar os modelos que queremos treinar com uma arquitetura de camadas. No contexto de regressão linear, temos uma arquitetura de apenas duas camadas:

<img src="book_imgs/modbackprop.png">

A nossa primeira camada é a linear, que multiplica os input pelos parâmetros e soma esse relutado. Matematicamente, nós podemos representar essa camada da seguinte forma:
$$f(\pmb{z}, \pmb{w}) = \pmb{z} \cdot \pmb{w} $$

A segunda camada é a nossa função custo. Ela tira a média da diferença entre o input e o valor alvo observado. Matematicamente, ela é definida da seguinte forma:

$$f(z) = \frac{1}{n} \sum (y-z)^2 $$

O que definimos acima é o que chamamos de forward pass das camadas. Para utilizar backpropagation nós ainda precisamos de mais dois mecanismos em cada camada. Em primeiro lugar, como dissemos na introdução, nós precisamos de algum mecanismo para repassar as derivadas de trás para frente. Isso será o backward pass. Matematicamente, tudo o que precisamos definir é a derivada da função da camada com respeito aos input da camada. Assim, no caso da camada linear teríamos

$$\frac{\partial f(\pmb{z})}{\partial \pmb{z}} = \pmb{w}$$

No caso da nossa camada de custo teríamos:

$$\frac{\partial f(\pmb{z})}{\partial \pmb{z}} =  -\frac{2}{n} \sum (y-z)$$

Por fim, o terceiro mecanismo que precisamos é uma forma de calcular a derivada da função da camada com respeito aos parâmetros da camada. Note que no caso de regressão linear, apenas a camada linear tem parâmetros. Assim, a derivada da camada linear com respeito aos parâmetros seria

$$\frac{\partial f(\pmb{z})}{\partial \pmb{w}} = \pmb{z}$$

Intuitivamente, o que queremos saber é como perturbações em $w$ afetam a nossa função custo $L(w)$, ou então, como $L(w)$ e $w$ variam juntos. Para fazer isso, nós vamos propagar as perturbações em $L(w)$ até que elas cheguem em $w$, ou seja, nós vamos ver como perturbações na camada de custo são reflexos de perturbações na camada linear e então veremos como que as perturbação na camada linear são reflexos de perturbações nos seus parâmetros. Assim, indiretamente, vamos saber como as perturbações nos parâmetros perturbam a função custo. Isso nos dará a direção de descida mais íngreme na função custo, ou a derivada da função custo com respeito aos parâmetros, que então será utilizada para otimização com gradiente descendente.


# Modularidade

Nos agora vamos mostrar que se definirmos camadas com esses dois ou três mecanismos será possível saber a derivada da função custo com respeito a qualquer parâmetro. Primeiro, vamos entender melhor a modularidade da camada. Primeiro, vamos adotar a seguinte convenção. No forward pass, uma camada $i$ qualquer recebe como inputs $z_i$ e produz como output $z_{i+1}$. Além disso, nos vamos definir a derivada da função custo com respeito ao input da camada $i$ como $\delta_i$. Com isso, podemos derivar o algoritmo de backpropagation apenas em termos de $z$ e $\delta$, o que tornará a matemática menos carregada e a regra mais geral.

Vamos agora voltar a nossa abstração de camadas. Aqui, eu utilizarei apenas 2 camadas porque esse é o número de camadas que precisamos no nosso contexto de regressão linear, mas o que será desenvolvido valerá para qualquer número de camadas. O que queremos construir pode ser resumido na seguinte imagem.

<img src="book_imgs/backpropreglin.png">

Acima, temos o forward pass na esquerda, onde começamos com $z_1$ e passamos adiante por duas funções, a linear e o erro quadrático médio, respectivamente. Do lado direito temos o backward pass, onde começamos com $\delta$ inciado sempre com 1, e propagamos as derivadas de trás para frente. Eu ainda não mostrei como faremos isso, mas é bom já ter em menta a imagem do que queremos construir. O que vou mostrar é que, para construir a arquitetura acima ou qualquer outra arquitetura, nós só precisamos implementar um módulo bem simples, da seguinte forma:


<img src="book_imgs/modulo.png">

Isto é, precisamos apenas construir módulos que aceitam $z_i$ como input e produzem $z_{i+1}$ como outpur e também aceitam $\delta_{i+1}$ e produzem $\delta_{i}$. Além disso, ocasionalmente esse módulos terão parâmetros, então precisaremos calcular a derivada do custo com respeito aos parâmetros.  

Ok. Vamos recapitular o que temos até aqui e o que falta definir. A única coisa que temos é um esqueleto de um módulo. Falta ainda definir como passar os $z$s adiante, como propagar os $\delta$s de trás para frente e falta ainda o mais importante: **achar as derivadas da função custo com respeito aos parâmetros** para poder aplicar gradiente descendente. No modulo acima, isso é representado pela seta que sai para a direita. Em certo sentido, parece que não fomos muito longe, mas veremos como é simples realizar essas três operações, uma vez que tenhamos definido o módulo acima. 

O primeiro mecanismo que falta definir é o forward pass. Ele é bastante simples e é definido da seguinte maneira:

$$z_{i+1} = f_i(z_i)$$

Sendo $f_i$ uma função diferenciável qualquer.

O próximo mecanismo é o backward pass. Ele é apenas um pouco mais complicado, mas nós conseguimos chegar nele facilmente a partir da definição de $\delta$ (lembre-se de que $\delta_i$ é a derivada da função custo com respeito ao input $z_i$ da camada):

$$\delta_i = \frac{\partial L}{\partial z_i} = \frac{\partial L}{\partial z_{i+1}} * \frac{\partial z_{i+1}}{\partial z_i}= \delta_{i+1} \frac{\partial z_{i+1}}{\partial z_i}$$

Com isso, podemos ver que $\delta_i$ é definido em termos do próximo $\delta$, assim, só precisamos iniciar o último $\delta$ para calcular todos eles de trás para frente. Bom, mas como $\delta_i$ é a derivada do input da camada com respeito à função custo e a última camada é o próprio custo, então o último $\delta$ só pode ser $1$! Por isso iniciamos ele dessa maneira na imagem acima.

Por fim, podemos definir o último mecanismo:

$$\frac{\partial L}{\partial w_i} = \frac{\partial L}{\partial z_{i+1}} * \frac{\partial z_{i+1}}{\partial w_i}= \delta_{i+1} \frac{\partial z_{i+1}}{\partial w_i}$$

OK. Chegamos em uma parte **muito importante**. A equação acima nos diz que a derivada da função custo com respeito aos parâmetros de qualquer camada pode ser definida em termos do $\delta$ da camada seguinte **e** da derivada do output da camada com respeito aos parâmetros da camada:

$$\frac{\partial L}{\partial w_i} = \delta_{i+1} \frac{\partial z_{i+1}}{\partial w_i}$$

Assim, podemos ver que para propagar as derivadas a única coisas que precisamos saber de fato são como definir os $\delta$s de cada camada e a derivada do output da camada com respeito aos parâmetros dessa mesma camada. Isso torna nosso trabalho extremamente mais simples, pois as camadas são funções relativamente simples. Além disso, essa modularidade nos permite implementar um tipo de camada e depois reutilizá-lo nas mais diversas arquiteturas possíveis. 

# Implementação

Com essa noção de modularidade, implementar gradiente descendente fica extremamente fácil. Em vez de ter que achar de uma vez a derivada da função custo com respeito aos parâmetros, **nós só precisamos empilhar camadas**. Além disso, precisamos que cada camada realize 2 ou 3 operações:  

1) O forward pass  
2) O backward pass   
3) *Se for o caso, calcular a derivada do output da camada com respeito aos parâmetros

Para mostrar isso, vamos simular dados como no tutorial de gradiente descendente:

In [1]:
import pandas as pd
import numpy as np
np.random.seed(0)
from matplotlib import pyplot as plt

dados = pd.DataFrame()
dados['x'] = np.linspace(-10,10,100)
dados['y'] = 5 + 3*dados['x'] + np.random.normal(0,3,100)

Agora, vamos implementar cada camada individualmente. A primeira delas é a camada linear. Recapitulando, para essa camada teremos o forward pass:

$$f(\pmb{z}_i, \pmb{w}) = \pmb{z}_i \cdot \pmb{w} = \pmb{z}_{i+1}$$

O backward pass:

$$\delta_i =  \delta_{i+1} \frac{\partial f(\pmb{z}_i)}{\partial \pmb{z}_i} = \delta_{i+1} \pmb{w}$$

E como a camada tem parâmetros, precisamos computar a derivada do custo com respeito aos parâmetros:

$$\frac{\partial L}{\partial \pmb{w}_i} = \delta_{i+1} \frac{\partial \pmb{z}_{i+1}}{\partial \pmb{w}_i} = \delta_{i+1} \pmb{z}_i$$

In [95]:
class linear(object):
    
    def __init__(self, W):
        self.W = W
    
    def forward(self, z_in):
        self.z_in = z_in
        self.z_out = np.dot(z_in, self.W)
        return self.z_out
    
    def backward(self, d_in):        
        self.dW = d_in.dot(self.z_in) / d_in.shape[0]
        self.d_out = d_in.reshape(-1,1).dot(self.W.reshape(1,-1))
        return self.d_out

Agora, vamos definir a segunda camada, que será o erro quadrático médio. Apenas recapitulando, o forward pass seria
$$f(\pmb{z}_i) = \frac{1}{n} \sum (\pmb{y}-\pmb{z}_i)^2 =\pmb{z}_{i+1}$$

O backward pass:

$$\delta_i =  \delta_{i+1} \frac{\partial f(\pmb{z}_i)}{\partial \pmb{z}_i} = -\delta_{i+1} \frac{2}{n} \sum (y-z)$$

Como vamos multiplicar o último $\delta$ pela taxa de aprendizado, não vamos nos preocupar com a constante $\frac{2}{n}$. Além disso, como essa é nossa última camada, nós já vamos definir o $\delta_{i+1}$ como sendo 1.

In [96]:
class sqrerror(object):
    
    def __init__(self, y):
        self.y = y
        
    def forward(self, z_in):
        self.z_in = z_in
        self.z_out = np.mean(np.square(z_in - self.y))
        return self.z_out
    
    def backward(self):
        self.d_out = -1 * (self.y - self.z_in)
        return self.d_out


Agora que temos todas as camadas implementadas, podemos empilhá-las para montar a arquitetura de regressão linear.

In [98]:
class linear_regr(object):
    
    def __init__(self, learning_rate=0.0001, training_iters=100):
        # define os hiper-parâmetros
        self.learning_rate = learning_rate 
        self.training_iters = training_iters

    def fit(self, X_train, y_train, plot=False):
        
        # formata os dados
        if len(X_train.values.shape) < 2:
            X_train = X_train.values.reshape(-1,1)
        X = np.insert(X_train, 0, 1, 1)
        
        # inicia os parâmetros
        self.w_hat = np.random.normal(0,1, size = X[0].shape)
        
        # constroi a arquitetura do modelo
        self.linear_layer = linear(self.w_hat) # camada linear
        self.loss = sqrerror(y_train) # camada de custo
        
        # loop de treino
        for _ in range(self.training_iters):
            
            # forward pass
            z2 = self.linear_layer.forward(X)
            self.loss.forward(z2)
            
            # backward pass
            d2 = self.loss.backward()
            self.linear_layer.backward(d2)
            
            # acha o gradiente
            gradient = self.linear_layer.dW
            gradient *= self.learning_rate # multiplica o gradiente pela taxa de aprendizado
                
            # atualiza os parâmetros
            self.w_hat -= gradient
                
        
    def predict(self, X_test):
        # formata os dados
        if len(X_test.values.shape) < 2:
            X = X_test.values.reshape(-1,1)
        X = np.insert(X, 0, 1, 1)
        
        return np.dot(X, self.w_hat) 
    
    
regr = linear_regr(learning_rate=0.05, training_iters=100)
regr.fit(dados['x'], dados['y'])
regr.w_hat

/Users/matheusfacure/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  del sys.path[0]


array([5.15421109, 2.95540097])

E prontinho. Temos a nossa regressão linear com backpropagation. Nós podemos conferir se ela produz bons resultados comparando com os parâmetros produzidos com a forma analítica:

In [12]:
from sklearn.linear_model import LinearRegression

regr = LinearRegression()
regr.fit(dados['x'].values.reshape(-1,1), dados['y'])
regr.intercept_, regr.coef_[0]


(5.179424046603455, 2.9554009703076836)